In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
from google_drive_downloader import GoogleDriveDownloader as gdd
def _download_data(download=True):
    if download:
        gdd.download_file_from_google_drive(
            file_id="1-e8PVHbFu_LaMKConDKeL7GPJdl4yZQB",
            dest_path="./data_temp/Microdato_Censo2017-Viviendas.zip",
            unzip=True
        )
    return True

In [3]:
_download_data(False)
df = pd.read_csv("./data_temp/Microdato_Censo2017-Viviendas.csv", sep=";", low_memory=False)

In [4]:
_drop = ["REGION", "PROVINCIA", "DC", "ZC_LOC", "ID_ZONA_LOC", "NVIV"]
_groupby = ["COMUNA", "AREA", "P01", "P02", "P03A", "P03B", "P03C", "P04", "P05"]

q = df.drop(columns=_drop).groupby(_groupby)
_ = pd.DataFrame({
    'CANT_VIV' : q.size(),
    'CANT_HOG' : q["CANT_HOG"].agg('sum'),
    "CANT_PER" : q["CANT_PER"].agg('sum')
}).reset_index()

_ = _.rename(columns = {
    "COMUNA": "comuna_id", 
    "AREA": "area_id", 
    "P01": "housing_type", 
    "P02": "housing_occupation", 
    "P03A": "walls_material", 
    "P03B": "ceiling_material", 
    "P03C": "floor_material", 
    "P04": "rooms",
    "P05": "water_source",
    "CANT_HOG": "cant_hog",
    "CANT_PER": "cant_per",
    "CANT_VIV": "cant_viv"
})

In [5]:
geo_datachile_id = pd.read_csv("https://raw.githubusercontent.com/datachile/datachile-etl/master/official_ids/2017_06_27_comunas_datachile_fixed.csv")
geo_datachile_id = geo_datachile_id.drop(columns=["region_id", "region_name", "comuna_tax_office_id"])

geo_datachile_id = geo_datachile_id.fillna(0).astype({"comuna_customs_id": "int"})
geo_datachile_id = geo_datachile_id.rename(columns={"comuna_customs_id": "comuna_id"})

In [6]:
_ = pd.merge(_, geo_datachile_id, on="comuna_id", how="inner")

In [7]:
_ = _.drop(columns=["comuna_id", "comuna_name"])

In [8]:
pd.DataFrame(_).to_csv("data_final/housing_census.csv", index=False)